In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score

In [2]:
df = pd.read_csv("Speeddating_Processed.csv")
df = df.drop(["Unnamed: 0", "has_null", "wave", "interests_correlate", "guess_prob_liked", "decision", "decision_o", "met"], axis=1)
df = df.drop([col for col in df if col.startswith("d_")], axis=1)
df = df.drop([col for col in df if "_o" in col], axis=1)

for ind, row in df.iterrows():
    if row["gender"] == "female":
        df.loc[ind, "gender"] = 1
    elif row["gender"] == "male":
        df.loc[ind, "gender"] = 0

df = df.join(pd.get_dummies(df.pop("race"), dtype=int))

# Field Column One Hot Encoded or Dropped
# df = df.join(pd.get_dummies(df.pop("field"), dtype=int))
df = df.drop("field", axis=1)

df.head()

,gender,age,samerace,importance_same_race,importance_same_religion,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,...,shopping,yoga,expected_happy_with_sd_people,like,match,Asian/Pacific Islander/Asian-American,Black/African American,European/Caucasian-American,Latino/Hispanic American,Other
0,1,21.0,0,2.0,4.0,15.0,20.0,20.0,15.0,15.0,...,8.0,1.0,3.0,7.0,0,1,0,0,0,0
1,1,21.0,0,2.0,4.0,15.0,20.0,20.0,15.0,15.0,...,8.0,1.0,3.0,7.0,0,1,0,0,0,0
2,1,21.0,0,2.0,4.0,15.0,20.0,20.0,15.0,15.0,...,8.0,1.0,3.0,7.0,1,1,0,0,0,0
3,1,21.0,0,2.0,4.0,15.0,20.0,20.0,15.0,15.0,...,8.0,1.0,3.0,6.0,1,1,0,0,0,0
4,1,21.0,0,2.0,4.0,15.0,20.0,20.0,15.0,15.0,...,8.0,1.0,3.0,6.0,0,1,0,0,0,0


In [3]:
tempDF = df.drop("match", axis=1)

scaler = StandardScaler()
scaled_arr = scaler.fit_transform(tempDF)
scaled_df = pd.DataFrame(scaled_arr, index=tempDF.index, columns=tempDF.columns).join(df.loc[:,"match"])

scaled_df

,gender,age,samerace,importance_same_race,importance_same_religion,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,...,shopping,yoga,expected_happy_with_sd_people,like,Asian/Pacific Islander/Asian-American,Black/African American,European/Caucasian-American,Latino/Hispanic American,Other,match
0,0.999294,-1.510659,-0.817314,-0.629416,0.121457,-0.597422,0.362857,-0.057670,-0.411495,0.700629,...,0.880210,-1.244794,-1.455416,0.465329,1.764456,-0.223574,-1.137174,-0.290812,-0.268795,0
1,0.999294,-1.510659,-0.817314,-0.629416,0.121457,-0.597422,0.362857,-0.057670,-0.411495,0.700629,...,0.880210,-1.244794,-1.455416,0.465329,1.764456,-0.223574,-1.137174,-0.290812,-0.268795,0
2,0.999294,-1.510659,-0.817314,-0.629416,0.121457,-0.597422,0.362857,-0.057670,-0.411495,0.700629,...,0.880210,-1.244794,-1.455416,0.465329,1.764456,-0.223574,-1.137174,-0.290812,-0.268795,1
3,0.999294,-1.510659,-0.817314,-0.629416,0.121457,-0.597422,0.362857,-0.057670,-0.411495,0.700629,...,0.880210,-1.244794,-1.455416,-0.086090,1.764456,-0.223574,-1.137174,-0.290812,-0.268795,1
4,0.999294,-1.510659,-0.817314,-0.629416,0.121457,-0.597422,0.362857,-0.057670,-0.411495,0.700629,...,0.880210,-1.244794,-1.455416,-0.086090,1.764456,-0.223574,-1.137174,-0.290812,-0.268795,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7074,-1.000707,-0.371125,1.223520,-0.982478,-0.943086,4.046668,-2.518729,-0.797401,-0.411495,-1.767049,...,0.496882,-0.506253,2.591509,-1.188927,-0.566747,-0.223574,0.879373,-0.290812,-0.268795,0
7075,-1.000707,-0.371125,-0.817314,-0.982478,-0.943086,4.046668,-2.518729,-0.797401,-0.411495,-1.767049,...,0.496882,-0.506253,2.591509,-2.291764,-0.566747,-0.223574,0.879373,-0.290812,-0.268795,0
7076,-1.000707,-0.371125,-0.817314,-0.982478,-0.943086,4.046668,-2.518729,-0.797401,-0.411495,-1.767049,...,0.496882,-0.506253,2.591509,-1.188927,-0.566747,-0.223574,0.879373,-0.290812,-0.268795,0
7077,-1.000707,-0.371125,-0.817314,-0.982478,-0.943086,4.046668,-2.518729,-0.797401,-0.411495,-1.767049,...,0.496882,-0.506253,2.591509,-0.637508,-0.566747,-0.223574,0.879373,-0.290812,-0.268795,0


In [4]:
X = scaled_df.drop("match", axis=1)
y = scaled_df["match"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [7]:
model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [8]:
preds = model.predict(X_test)
acc = accuracy_score(y_test, preds)
f1 = f1_score(y_test, preds)
rec = recall_score(y_test, preds)

print("LDA Accuracy:", acc)
print("LDA F1_Score:", f1)
print("LDA Recall:", rec)
#acc, f1, rec

(0.8152542372881356, 0.18858560794044663, 0.11275964391691394)